# SENTIMENT ANALYSIS ON HOTEL RATING

### Hotel Name: Occidental Grand Xcaret & Royal Club & HotelID: 260444

In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from keras import models
from keras.layers import Dense

In [ ]:
# Load Data set
df = pd.read_csv('Reviews.csv') # Converted 260444.json to Reviews.csv by using mongoDB compass
df.head()

In [ ]:
# Required dataset for training
""" We need only two columns Content and Ratings.Overall """
reqData = df[['Content', 'Ratings.Overall']]
reqData.head()

In [ ]:
# Data Exploration
reqData.info()
reqData.describe()

In [ ]:
# Check For Null values
""" There is no null values """
reqData.isnull().sum()


In [ ]:
# Check for unique values
reqData['Ratings.Overall'].unique()

In [ ]:
# Data Visualization
""" We can see that most of the people give 5 Rating """
reqData['Ratings.Overall'].value_counts()
num = [5,4,3,2,1]
plt.bar(num, reqData['Ratings.Overall'].value_counts())


In [ ]:
reqData.Content[2]

### Data Preprocessing / Cleaning

In [ ]:
# Data Preprocessing / Cleaning
sw = set(stopwords.words('english'))
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
ps.stem('jumping')

In [ ]:
# Create a function to clean data
def cleanText(sample):
    sample = sample.lower()
    # Remove punctuations
    sample = re.sub("[^a-zA-Z]+", ' ', sample)
    # Tokenizing sample (split every word)
    sample = sample.split()
    # Remove stopwords
    sample = [s for s in sample if s not in sw] # List comprehension
    # Stemmimg (convert words to root words: jumping-jump) 
    sample = [ps.stem(s) for s in sample]
    # Lemmatization it reduces size of vocabolary.
    sample = [lemmatizer.lemmatize(s) for s in sample]
    # Joins words
    sample = " ".join(sample)
       
    return sample

In [ ]:
cleanText(reqData.Content[2])

In [ ]:
# Apply cleanText function to DataFrame
reqData['CleanedContent'] = reqData['Content'].apply(cleanText)
reqData.head()

corpus = reqData['CleanedContent'].values #  Convert to Array
y = reqData['Ratings.Overall']

In [ ]:
# Convert Text data to matrix formate or tabular formate
cv = CountVectorizer(ngram_range=(1, 3), max_df=0.5, max_features=30000)
tfidf = TfidfTransformer()

X = cv.fit_transform(corpus)
print(X[0])

X = tfidf.fit_transform(X)
print(X[0])

### Create  a Artificial Neural Network

In [ ]:
# Create  Artificial Neural Network
model = models.Sequential([
    Dense(128, input_shape=(2020,30000,), activation="relu"),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(16, activation="selu"),
    Dense(6, activation="softmax"),
    ])

model.summary()

In [ ]:
#  Compile Model
model.compile(
    optimizer="rmsprop",
    loss = "sparse_categorical_crossentropy",
    metrics = ['accuracy'],
    )

In [ ]:
# Split data into train, test and validation
xval = X[:500]
xtest = X[500:900]
xtrain = X[900:]

yval = y[:500]
ytest = y[500:900]
ytrain = y[900:]

In [ ]:
# Converted data into Array
xtrainArr = xtrain.toarray()
xtestArr = xtest.toarray()
xvalArr = xval.toarray()

ytrainArr = np.array(y[900:])
yvalArr = np.array(y[:500])
ytestArr = np.array(y[500:900])

In [ ]:
# Train neural network
hist = model.fit(xtrainArr, ytrainArr, batch_size=16, epochs=2, validation_data=(xvalArr, yvalArr))

In [ ]:
result = hist.history

In [ ]:
# Training Visualization
""" Here we can adjust our epochs values to overcome model overfitting"""
plt.plot(result['val_accuracy'], label='Val_acc')
plt.plot(result['accuracy'], label="train_acc")
plt.legend()
plt.show()

In [ ]:
plt.plot(result['val_loss'], label='Val_loss')
plt.plot(result['loss'], label="train_los")
plt.legend()
plt.show()

In [ ]:
# Evaluate data
model.evaluate(xvalArr, yvalArr)

In [ ]:
# Test Our Model
yPred = model.predict(xtestArr)

In [ ]:
# Create a Comperision DataFrame
ypreValues =[]
for i in range(0, 400): 
    val = np.argmax(yPred[i])
    ypreValues.append(val)
    
yPredArr = np.array(ypreValues).reshape(-1, 1)
ytestArr = ytestArr.reshape(-1, 1)
ids = np.arange(400).reshape(-1, 1)
finalMatrix = np.hstack((ids, ytestArr, yPredArr))

In [ ]:
# Create a final result csv file
final =pd.DataFrame(finalMatrix, columns=['Id','Original_Value','Predicted_value'])  
# final.to_csv('Rating_Comperision.csv', index=False)----Already created